In [1]:
import argparse
import os
import os.path as osp
import shutil
import time
import warnings

import mmcv
import torch
from mmcv import Config
from mmcv.cnn.utils import revert_sync_batchnorm
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmcv.runner import (get_dist_info, init_dist, load_checkpoint,
                         wrap_fp16_model)
from mmcv.utils import DictAction

from mmseg import digit_version
from mmseg.apis import multi_gpu_test, single_gpu_test
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.utils import setup_multi_processes

/opt/conda/envs/mm_s/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# config 파일과 work_dir, epoch(model)을 설정해주세요.
cfg = Config.fromfile('/opt/ml/level2_semanticsegmentation_cv-level2-cv-01/mmsegmentation/_trashsegmentation/convnext/upernet_convnext_xlarge_fp16_640x640_160k_ade20k.py')
cfg.work_dir = '/opt/ml/level2_semanticsegmentation_cv-level2-cv-01/mmsegmentation/work_dirs/upernet_convnext_xlarge_fp16_640x640_160k_ade20k'
epoch = 'best_mIoU_iter_40000'


cfg.seed=2022
cfg.gpu_ids = [1]
cfg.model.train_cfg = None
cfg.data.test.test_mode = True

# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=4,
        dist=False,
        shuffle=False)

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2022-12-29 09:56:02,216 - mmseg - INFO - Loaded 819 images


AttributeError: 'ConfigDict' object has no attribute 'workers_per_gpu'